In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from fuzzywuzzy import fuzz
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
stopword=nltk.corpus.stopwords.words('english')
import string
string.punctuation

In [ ]:
import gensim
from gensim.models import Word2Vec
    
model = gensim.models.KeyedVectors.load_word2vec_format('../input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
df=pd.read_csv('../input/quora-question-pairs/train.csv')
print(df.shape)
df.head()

In [ ]:
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)
data=df[0:10000]
data.head()

In [ ]:
def rem_p(text):
    text_nopu="".join([char for char in text if char not in string.punctuation])
    return text_nopu
data['question1']=data['question1'].apply(lambda x:rem_p(x))
data['question2']=data['question2'].apply(lambda x:rem_p(x))
data.head(20)

In [ ]:
def dis(question1,question2):
    model.init_sims(replace=True)
    distance = model.wmdistance(question1, question2)
    return distance
data['wmdistance']=data.apply(lambda x: dis(x['question1'], x['question2']), axis=1)
data.head()

In [ ]:
data.head(20)

In [ ]:
data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
data['diff_len'] = data.len_q1 - data.len_q2
data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))
data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
data.head(10)

In [ ]:
data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data.head()

In [ ]:
data.to_csv("data.csv")